## 9주 기념축사 TF-IDF
2019.10.21일 '제74주년 경찰의 날 기념식 축사' 전문을 변환하세요.
전문은 http://www.korea.kr/archive/speechView.do?newsId=132031636에서 읽을 수 있고, 해당 사이트에서 텍스트만 파일로 저장해서 사용한다.

* 1) DataFrame 생성
* 2) 단어로 분리해서, 출력
* 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성
* 4) 불용어 제거하고, 출력
* 5) TF-IDF를 계산하고, 출력
* 6) TF-IDF 컬럼을 features로 구성, 출력

#### 1. dataFrame 생성하기

In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/제74주년_경찰의_날_기념식_축사.txt")

Out[68]: [FileInfo(path='dbfs:/FileStore/tables/제74주년_경찰의_날_기념식_축사.txt', name='제74주년_경찰의_날_기념식_축사.txt', size=5050)]

In [0]:
import os
import pyspark

myConf=pyspark.SparkConf()
spark=pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

# 경로를 pas에 저장한 후, pas를 통해 가져온 데이터를 rdd로 생성한다. 
# rdd=spark.sparkContext.textFile(os.path.join("/FileStore/tables/","제74주년_경찰의_날_기념식_축사.txt"))
pas = os.path.join("/FileStore/tables/","제74주년_경찰의_날_기념식_축사.txt")
rdd=spark.sparkContext.textFile(pas).map(lambda x: x.split(';'))
rdd.take(1)

Out[69]: [['존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.']]

In [0]:
df=spark.createDataFrame(rdd, ['sent'])
df.show(truncate=True)

+----------------------------------+
 sent|
+----------------------------------+
 존경하는 국민 여러분, 경찰관 ...|
 |
 국민의 안전을 위해 밤낮없이 애...|
 |
오늘 홍조근정훈장을 받으신 중앙...|
 |
 사랑하는 경찰관 여러분,|
 |
여러분의 헌신적 노력으로 우리의...|
 |
 치안의 개선은 국민의 체감으로 ...|
 |
 한국을 찾는 외국 관광객들도 우...|
 |
 올해는 ‘경찰의 날’에 맞춰 국...|
 |
 자랑스러운 경찰관 여러분,|
 |
 경찰헌장은 “나라와 겨레를 위하...|
 |
+----------------------------------+
only showing top 20 rows

#### 2. 단어로 분리해서 출력

In [0]:
# Tokenizer로 white space 분리하기
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(df)
tokDf.show()

+----------------------------------+--------------------------------+
 sent| words|
+----------------------------------+--------------------------------+
 존경하는 국민 여러분, 경찰관 ...| [존경하는, 국민, 여러분,, ...|
 | []|
 국민의 안전을 위해 밤낮없이 애...| [국민의, 안전을, 위해, 밤낮...|
 | []|
오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
 | []|
 사랑하는 경찰관 여러분,| [사랑하는, 경찰관, 여러분,]|
 | []|
여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
 | []|
 치안의 개선은 국민의 체감으로 ...| [치안의, 개선은, 국민의, 체...|
 | []|
 한국을 찾는 외국 관광객들도 우...| [한국을, 찾는, 외국, 관광객...|
 | []|
 올해는 ‘경찰의 날’에 맞춰 국...| [올해는, ‘경찰의, 날’에, ...|
 | []|
 자랑스러운 경찰관 여러분,| [자랑스러운, 경찰관, 여러분,]|
 | []|
 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|
 | []|
+----------------------------------+--------------------------------+
only showing top 20 rows

In [0]:
# RegexTokenizer로 분리하기 (정규표현식을 사용하여 분리 및 특정 패턴 추출)
from pyspark.ml.feature import RegexTokenizer
re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")
reDf=re.transform(df)
reDf.show()

+----------------------------------+--------------------------------+
 sent| wordsReg|
+----------------------------------+--------------------------------+
 존경하는 국민 여러분, 경찰관 ...| [존경하는, 국민, 여러분,, ...|
 | []|
 국민의 안전을 위해 밤낮없이 애...| [국민의, 안전을, 위해, 밤낮...|
 | []|
오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
 | []|
 사랑하는 경찰관 여러분,| [사랑하는, 경찰관, 여러분,]|
 | []|
여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
 | []|
 치안의 개선은 국민의 체감으로 ...| [치안의, 개선은, 국민의, 체...|
 | []|
 한국을 찾는 외국 관광객들도 우...| [한국을, 찾는, 외국, 관광객...|
 | []|
 올해는 ‘경찰의 날’에 맞춰 국...| [올해는, ‘경찰의, 날’에, ...|
 | []|
 자랑스러운 경찰관 여러분,| [자랑스러운, 경찰관, 여러분,]|
 | []|
 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|
 | []|
+----------------------------------+--------------------------------+
only showing top 20 rows

#### 3. 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [0]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

# 불용어 리스트 가져오기
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

In [0]:
# 내가 지정한 불용어 리스트를 추가해서 재설정하기
_mystopwords=[u"그",u"에", u"우리", u"등", u"것이", u"그러나", u"우리", u"우리의", u"그리고", u"또한", u"여러분의", u"여러분,", u"위해", u"비롯한", u"참", u"이와 같은"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

Out[74]: StopWordsRemover_56b2d98f6da4

In [0]:
# 내가 추가한 불용어가 있는지 확인하기
for e in stop.getStopWords():
    print (e, end="/")

i/me/my/myself/we/our/ours/ourselves/you/your/yours/yourself/yourselves/he/him/his/himself/she/her/hers/herself/it/its/itself/they/them/their/theirs/themselves/what/which/who/whom/this/that/these/those/am/is/are/was/were/be/been/being/have/has/had/having/do/does/did/doing/a/an/the/and/but/if/or/because/as/until/while/of/at/by/for/with/about/against/between/into/through/during/before/after/above/below/to/from/up/down/in/out/on/off/over/under/again/further/then/once/here/there/when/where/why/how/all/any/both/each/few/more/most/other/some/such/no/nor/not/only/own/same/so/than/too/very/s/t/can/will/just/don/should/now/i'll/you'll/he'll/she'll/we'll/they'll/i'd/you'd/he'd/she'd/we'd/they'd/i'm/you're/he's/she's/it's/we're/they're/i've/we've/you've/they've/isn't/aren't/wasn't/weren't/haven't/hasn't/hadn't/don't/doesn't/didn't/won't/wouldn't/shan't/shouldn't/mustn't/can't/couldn't/cannot/could/here's/how's/let's/ought/that's/there's/what's/when's/where's/who's/why's/would/그/에/우리/등/것이/그러나/우리/우리의/그리고/또한/여러분의/여러분,/위해/비롯한/참/이와 같은/

#### 4. 불용어 제거하고, 출력

In [0]:
# transformer로 불용어를 제거하기
stopDf=stop.transform(reDf)
stopDf.show()

+----------------------------------+--------------------------------+--------------------------------+
 sent| wordsReg| nostops|
+----------------------------------+--------------------------------+--------------------------------+
 존경하는 국민 여러분, 경찰관 ...| [존경하는, 국민, 여러분,, ...| [존경하는, 국민, 경찰관, 일...|
 | []| []|
 국민의 안전을 위해 밤낮없이 애...| [국민의, 안전을, 위해, 밤낮...| [국민의, 안전을, 밤낮없이, ...|
 | []| []|
오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
 | []| []|
 사랑하는 경찰관 여러분,| [사랑하는, 경찰관, 여러분,]| [사랑하는, 경찰관]|
 | []| []|
여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...| [헌신적, 노력으로, 치안은, ...|
 | []| []|
 치안의 개선은 국민의 체감으로 ...| [치안의, 개선은, 국민의, 체...| [치안의, 개선은, 국민의, 체...|
 | []| []|
 한국을 찾는 외국 관광객들도 우...| [한국을, 찾는, 외국, 관광객...| [한국을, 찾는, 외국, 관광객...|
 | []| []|
 올해는 ‘경찰의 날’에 맞춰 국...| [올해는, ‘경찰의, 날’에, ...| [올해는, ‘경찰의, 날’에, ...|
 | []| []|
 자랑스러운 경찰관 여러분,| [자랑스러운, 경찰관, 여러분,]| [자랑스러운, 경찰관]|
 | []| []|
 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...| [경찰헌장은, “나라와, 겨레를...|
 | []| []|
+----------------------------------+--------------------------------+--------------------------------+
only showing top 20 rows

##### + CountVetorizer하기

In [0]:
# Spark CountVectorizer, CountVectorizer는 단어를 분리하고 나서, 빈도를 계산할 수 있음. 결과는 단어별 단어빈도 TF. 제거해야할 단어, 즉 문서에 사용된 빈도 document frequency를 minDF, maxDF를 통해 설정할 수 있음. 소수점을 사용하면, 비율, 즉 '사용된 문서 수/전체 문서 수'를 의미함.

from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=350, minDF=1.0)
cvModel = cv.fit(stopDf)

In [0]:
cvDf = cvModel.transform(stopDf)

# 일부 컬럼만 선택해서 결과 보기 
cvDf.select('sent','nostops','cv').show() 

+----------------------------------+--------------------------------+--------------------+
 sent| nostops| cv|
+----------------------------------+--------------------------------+--------------------+
 존경하는 국민 여러분, 경찰관 ...| [존경하는, 국민, 경찰관, 일...|(346,[3,28,67,82,...|
 | []| (346,[],[])|
 국민의 안전을 위해 밤낮없이 애...| [국민의, 안전을, 밤낮없이, ...|(346,[1,3,7,9,16,...|
 | []| (346,[],[])|
오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|(346,[1,7,9,16,27...|
 | []| (346,[],[])|
 사랑하는 경찰관 여러분,| [사랑하는, 경찰관]|(346,[3,338],[1.0...|
 | []| (346,[],[])|
여러분의 헌신적 노력으로 우리의...| [헌신적, 노력으로, 치안은, ...|(346,[69,74,76,77...|
 | []| (346,[],[])|
 치안의 개선은 국민의 체감으로 ...| [치안의, 개선은, 국민의, 체...|(346,[1,2,10,40,4...|
 | []| (346,[],[])|
 한국을 찾는 외국 관광객들도 우...| [한국을, 찾는, 외국, 관광객...|(346,[4,8,22,33,7...|
 | []| (346,[],[])|
 올해는 ‘경찰의 날’에 맞춰 국...| [올해는, ‘경찰의, 날’에, ...|(346,[6,11,14,28,...|
 | []| (346,[],[])|
 자랑스러운 경찰관 여러분,| [자랑스러운, 경찰관]|(346,[3,234],[1.0...|
 | []| (346,[],[])|
 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|(346,[0,13,15,90,...|
 | []| (346,[],[])|
+----------------------------------+--------------------------------+--------------------+
only showing top 20 rows

In [0]:
# CountVectorizer에서 사용된 단어 목록을 출력(sparse vector의 컬럼 개수와 동일함)
cvModel.vocabulary

Out[79]: ['경찰은',
 '국민의',
 '있습니다.',
 '경찰관',
 '합니다.',
 '경찰의',
 '것입니다.',
 '여러분께',
 '가장',
 '드립니다.',
 '역대',
 '함께',
 '검찰과',
 '경찰헌장은',
 '더',
 '겨레를',
 '감사를',
 '안전을',
 '경찰,',
 '광복',
 '정부는',
 '법',
 '외국',
 '감사합니다.',
 '경찰이',
 '먼저',
 '국민과',
 '받으신',
 '‘경찰의',
 '임시정부',
 '대한민국',
 '특히',
 '노력하고',
 '경찰을',
 '바랍니다.',
 '엄정하고',
 '고맙습니다.',
 '법을',
 '경찰',
 '공정한',
 '최고를',
 '안보를',
 '범죄안전도는',
 '없이',
 '될',
 '74.5점으로',
 '확립하고자',
 '개선하기',
 '후로도',
 '만들고',
 '치안',
 '관계자',
 '15만',
 '광주남부경찰서',
 '피를',
 '감사의',
 '주시기',
 '근무여건을',
 '여러분을',
 '관헌에게',
 '돌아보며',
 '경찰위원장님,',
 '공정해야',
 '강영규',
 '세계에',
 '그러자면',
 '알려지게',
 '돌',
 '수사권',
 '지난해',
 '물음에',
 '위로의',
 '기대로',
 '김구',
 '기간',
 '평가합니다.',
 '좋아졌습니다.',
 '범죄',
 '부실하거나',
 '체감으로',
 '치안의',
 '수사로',
 '일흔네',
 '기록했습니다.',
 '갖추고',
 '일제와',
 '치안을',
 '경우회장님과',
 '누구도',
 '잃었습니다.',
 '다짐으로',
 '없는',
 '삼으려',
 '‘나라와',
 '비해',
 '의로운',
 '마땅합니다.',
 '본연의',
 '발전이',
 '발전해',
 '요인들을',
 '시민',
 '이후',
 '수반하는',
 '치안이',
 '관광객들은',
 '애쓰시는',
 '공권력이',
 '집행하며',
 '문재인',
 '탄생했습니다.',
 '개선은',
 '맞춰',
 '가시기',
 '15.1%',
 '던지고,',
 '공식',
 '초대',
 '이미',
 '과제가',
 '충성하는',
 '집행해야',
 '최중락님께',
 '누구에게나',
 '2015년에',
 '임무를',
 '드리기도',
 '위하여',
 '18.2%',
 '일본',
 '지금',
 '올해는',
 '마음을',
 '들어',
 '각별한',
 '공정하게',
 '걱정을',
 '헌신에',
 '보호했습니다.',
 '인권의',
 '늘렸고,',
 '공권력을',
 '전국의',
 '앞으로도',
 '길을',
 '위한',
 '수사',
 '인천시의회',
 '없지는',
 '한때',
 '7년',
 '모범으로',
 '다한다는',
 '준수해야',
 '수',
 '큰',
 '말씀을',
 '치안은',
 '존경하는',
 '시작합니다.',
 '길로',
 '널리',
 '의사',
 '민갑룡',
 '뿌리는',
 '나타나고',
 '8,572명을',
 '집행으로',
 '지키고',
 '밤낮없이',
 '거듭나려고',
 '“나라와',
 '준비하신',
 '전몰·순직',
 '감소했습니다.',
 '걸으려',
 '인권을',
 '그리하여',
 '지탄을',
 '박정훈',
 '일선',
 '입법을',
 '충원을',
 '스스로도',
 '불공정한',
 '개발도상국들은',
 '고귀한',
 '오늘',
 '올해',
 '응징하며,',
 '주기를',
 '근정포장을',
 '흔들림',
 '무리하게',
 '치안감님,',
 '미룰',
 '내빈',
 '영웅으로',
 '못하는',
 '체감안전도는',
 '서울국제경찰청장회의가',
 '군림하지',
 '추서되신',
 '80점을',
 '발전과',
 '절제하며',
 '국회에서',
 '경찰개혁은',
 '위에',
 '처음으로',
 '대해',
 '함께하신',
 '논의되고',
 '찾는',
 '소망합니다.',
 '행정안전부',
 '관광객들도',
 '과거를',
 '날’입니다.',
 '선생과',
 '수상자',
 '국가에',
 '않았습니다.',
 '싸웠습니다.',
 '행사하는',
 '경의를',
 '시대적',
 '유가족',
 '경찰청장님과',
 '헌신적',
 '경찰관이',
 '축하와',
 '사망자는',
 '자치경찰제',
 '자랑스러운',
 '법치주의를',
 '충성’의',
 '전합니다.',
 '날’에',
 '개혁을',
 '국군과',
 '정부',
 '좋았다고',
 '중앙경찰학교장',
 '나창헌,',
 '앞장서서',
 '응답했습니다.',
 '따뜻한',
 '이용범',
 '충성”을',
 '경찰청장님,',
 '줄었습니다.',
 '선진경찰로',
 '좋았느냐는',
 '치안산업',
 '제약을',
 '다짐합니다.',
 '검찰개혁과',
 '노력으로',
 '전문성을',
 '적도',
 '표합니다.',
 '왔습니다.',
 '폭탄을',
 '국민이',
 '유상근',
 '수행하는',
 '조정과',
 '됩니다.',
 '좋게',
 '같은',
 '매듭지어',
 '전쟁에서',
 '개혁위원회를',
 '중립성,',
 '열립니다.',
 '나석주,',
 '넘었습니다.',
 '밀정을',
 '이은정',
 '거듭',
 '김동현',
 '국민은',
 '나섰습니다.',
 '경무국장',
 '닿아',
 '임시정부에',
 '전해드립니다.',
 '직후에',
 '백범',
 '발생은',
 '계속할',
 '무기력한',
 '6·25전쟁에서도',
 '조속히',
 '노력해',
 '진영',
 '경우에는',
 '한국을',
 '더욱',
 '장관님,',
 '사랑을',
 '국회가',
 '도입이',
 '무엇이',
 '받은',
 '희생에',
 '응원할',
 '1만여',
 '경찰로',
 '근무환경을',
 '헌장처럼',
 '국가의',
 '국민',
 '국민께',
 '동시에',
 '흘렸습니다.',
 '상반기',
 '경찰관들의',
 '차일혁,',
 '행사를',
 '잘못도',
 '했습니다.',
 '개선하겠습니다.',
 '경찰님들을',
 '의장님,',
 '연속으로',
 '공정성,',
 '주목하고',
 '도왔습니다.',
 '교통사고',
 '임정',
 '훼손했습니다.',
 '국제치안산업박람회와',
 '목숨을',
 '자체개혁에',
 '엄격히',
 '명의',
 '사랑하는',
 '홍조근정훈장을',
 '됐습니다.',
 '검경',
 '충실히',
 '1945년',
 '경감님을',
 '한국의']

#### 5. TF-IDF를 계산하고, 출력

In [0]:
# HashingTF는 단어집합을 워드벡터 word vector로 변환하는데, 해시함수를 사용해서 단어에 해당하는 일련번호를 결정함
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)
# hashDf.select("nostops", "hash").show(truncate=False)

In [0]:
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [0]:
for e in idfDf.select("nostops","hash").take(10):
    print(e)

Row(nostops=['존경하는', '국민', '경찰관', '일흔네', '돌', '‘경찰의', '날’입니다.'], hash=SparseVector(262144, {162: 1.0, 80732: 1.0, 127225: 1.0, 142775: 1.0, 160086: 1.0, 172380: 1.0, 254275: 1.0}))
Row(nostops=[], hash=SparseVector(262144, {}))
Row(nostops=['국민의', '안전을', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'], hash=SparseVector(262144, {5341: 1.0, 6304: 1.0, 30732: 1.0, 39431: 1.0, 43098: 1.0, 49855: 1.0, 51468: 1.0, 63600: 1.0, 75300: 1.0, 77757: 1.0, 96799: 1.0, 110980: 1.0, 123553: 1.0, 160081: 1.0, 160086: 1.0, 167255: 1.0, 178931: 1.0, 208192: 1.0, 217323: 2.0, 257249: 1.0, 261393: 1.0}))
Row(nostops=[], hash=SparseVector(262144, {}))
Row(nostops=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'], hash=SparseVector(262144, {29823: 1.0, 32228: 1.0, 36822: 1.0, 39431: 1.0, 41144: 1.0, 49855: 1.0, 61014: 1.0, 61103: 1.0, 64713: 1.0, 72971: 1.0, 89227: 1.0, 118725: 2.0, 153204: 1.0, 167255: 1.0, 178229: 1.0, 185989: 1.0, 206065: 1.0, 217323: 1.0, 235673: 1.0, 237148: 1.0, 248593: 1.0, 249689: 1.0, 251574: 1.0, 254458: 1.0}))
Row(nostops=[], hash=SparseVector(262144, {}))
Row(nostops=['사랑하는', '경찰관'], hash=SparseVector(262144, {160086: 1.0, 220702: 1.0}))
Row(nostops=[], hash=SparseVector(262144, {}))
Row(nostops=['헌신적', '노력으로', '치안은', '좋아졌습니다.', '지난해', '범죄', '발생은', '2015년에', '비해', '15.1%', '줄었습니다.', '같은', '기간', '교통사고', '사망자는', '18.2%', '감소했습니다.'], hash=SparseVector(262144, {11202: 1.0, 28414: 1.0, 51063: 1.0, 53633: 1.0, 63981: 1.0, 80937: 1.0, 84279: 1.0, 92349: 1.0, 93327: 1.0, 108997: 1.0, 110381: 1.0, 155248: 1.0, 170244: 1.0, 197092: 1.0, 199006: 1.0, 211370: 1.0, 251648: 1.0}))
Row(nostops=[], hash=SparseVector(262144, {}))

#### 6. TF-IDF 컬럼을 features로 구성, 출력

In [0]:
idfDf.printSchema()

root
-- sent: string (nullable = true)
-- wordsReg: array (nullable = true)
 |-- element: string (containsNull = true)
-- nostops: array (nullable = true)
 |-- element: string (containsNull = true)
-- hash: vector (nullable = true)
-- idf: vector (nullable = true)

In [0]:
idfDf=idfDf.withColumnRenamed('nostops', 'label').withColumnRenamed('hash', 'features')

In [0]:
for e in idfDf.select("label","features").take(10):
    print(e)

Row(label=['존경하는', '국민', '경찰관', '일흔네', '돌', '‘경찰의', '날’입니다.'], features=SparseVector(262144, {162: 1.0, 80732: 1.0, 127225: 1.0, 142775: 1.0, 160086: 1.0, 172380: 1.0, 254275: 1.0}))
Row(label=[], features=SparseVector(262144, {}))
Row(label=['국민의', '안전을', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'], features=SparseVector(262144, {5341: 1.0, 6304: 1.0, 30732: 1.0, 39431: 1.0, 43098: 1.0, 49855: 1.0, 51468: 1.0, 63600: 1.0, 75300: 1.0, 77757: 1.0, 96799: 1.0, 110980: 1.0, 123553: 1.0, 160081: 1.0, 160086: 1.0, 167255: 1.0, 178931: 1.0, 208192: 1.0, 217323: 2.0, 257249: 1.0, 261393: 1.0}))
Row(label=[], features=SparseVector(262144, {}))
Row(label=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'], features=SparseVector(262144, {29823: 1.0, 32228: 1.0, 36822: 1.0, 39431: 1.0, 41144: 1.0, 49855: 1.0, 61014: 1.0, 61103: 1.0, 64713: 1.0, 72971: 1.0, 89227: 1.0, 118725: 2.0, 153204: 1.0, 167255: 1.0, 178229: 1.0, 185989: 1.0, 206065: 1.0, 217323: 1.0, 235673: 1.0, 237148: 1.0, 248593: 1.0, 249689: 1.0, 251574: 1.0, 254458: 1.0}))
Row(label=[], features=SparseVector(262144, {}))
Row(label=['사랑하는', '경찰관'], features=SparseVector(262144, {160086: 1.0, 220702: 1.0}))
Row(label=[], features=SparseVector(262144, {}))
Row(label=['헌신적', '노력으로', '치안은', '좋아졌습니다.', '지난해', '범죄', '발생은', '2015년에', '비해', '15.1%', '줄었습니다.', '같은', '기간', '교통사고', '사망자는', '18.2%', '감소했습니다.'], features=SparseVector(262144, {11202: 1.0, 28414: 1.0, 51063: 1.0, 53633: 1.0, 63981: 1.0, 80937: 1.0, 84279: 1.0, 92349: 1.0, 93327: 1.0, 108997: 1.0, 110381: 1.0, 155248: 1.0, 170244: 1.0, 197092: 1.0, 199006: 1.0, 211370: 1.0, 251648: 1.0}))
Row(label=[], features=SparseVector(262144, {}))